<a href="https://colab.research.google.com/github/yalopez84/Goog-Negative-Sampling/blob/master/GoodNegativeSampling_Freebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from tqdm import tqdm, trange
import random
import csv
from time import time
import pdb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir="/content/drive/MyDrive/NegativeStrategies/GoodNegativeSampling/FB13/"
os.chdir(data_dir)


In [ ]:
class InputExample(object):    
    def __init__(self, guid, text_a, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

In [ ]:
class Triple(object):
    def __init__(self, guid, subject , predicate , obj, label):
        self.guid=guid
        self.subject=subject
        self.predicate=predicate
        self.obj=obj
        self.label=label

In [ ]:
class Relation_Dom_Ran(object):
    
    def __init__(self, relation, dom, rang, hpt=0,tph=0,pr=0):
        self.relation = relation
        self.dom = dom
        self.rang = rang
        self.hpt = hpt  
        self.tph=tph
        self.pr=pr

In [ ]:
def CalcularBernoulliDistribution(relations=None, tripletas=None):
    
    for relation in relations:
        listah=[]
        listat=[]
        canth,cantt,hpt,tph,pr=0,0,0,0,0       
        for triple in tripletas:
            temp=triple.split('\t')
            if temp[1]==relation.relation:
                listah.append(temp[0])
                listat.append(temp[2])
        seth=set(listah)
        sett=set(listat) 
        canth= len(seth)  
        cantt= len(sett)
        hpt=canth/cantt
        tph=cantt/canth
        pr=tph/(tph+hpt)        
        relation.hpt=hpt
        relation.tph=tph
        relation.pr=pr
    return relations

In [ ]:
def validarTripleta(sujeto=None, dom=None, obj=None, ran=None, tripletas=None, relations=None):
    
    if sujeto!=None:
        if dom!=None:
            for triple in tripletas:
                temp=triple.split('\t')
                if temp[0]==sujeto:
                    doms=[relation.dom for relation in relations if relation.relation==temp[1] ]
                    if len(doms)>0:
                        if dom==doms[0]:
                            return True
                elif temp[2]==sujeto:
                    rangs=[relation.rang for relation in relations if relation.relation==temp[1] ]
                    if len(rangs)>0:
                        if dom==rangs[0]:
                            return True
    elif obj!=None:
        if ran!=None:
            for triple in tripletas:
                temp=triple.split('\t')
                if temp[0]==obj:
                    doms=[relation.dom for relation in relations if relation.relation==temp[1] ]
                    if len(doms)>0:
                        if ran==doms[0]:
                            return True
                elif temp[2]==obj:
                    rangs=[relation.rang for relation in relations if relation.relation==temp[1] ]
                    if len(rangs)>0:
                        if ran==rangs[0]:
                            return True
    return False 

In [ ]:
class DataProcessor(object):
    def get_train_examples(self, data_dir):
        raise NotImplementedError()
    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []            
            for line in reader:
                lines.append(line)  
            return lines  

In [ ]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train_reduced_11082.tsv")), "train", data_dir)
        
    def _create_examples(self, lines, set_type, data_dir):       

        examples = [] 
         
        train_with_corrupts=[]  

        examples_file=os.path.join(data_dir, "train_reduced_11082_13050_neg_and_descrip_goodnegatives.tsv")
        train_with_corrupts_file=os.path.join(data_dir, "train_reduced_11082_13050_neg_goodnegatives.tsv")
        
        #pdb.set_trace() 
        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r', encoding="utf-8") as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]      
        entities = list(ent2text.keys())

              
        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r', encoding="utf-8") as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')                
                rel2text[temp[0]] = temp[1]       
        lines_str_set = set(['\t'.join(line) for line in lines])
        
        
        #GoodNegativeSampler with Bernoulli
        relations_dom_ran=[] 
        with open(os.path.join(data_dir, "relations_dom_ran.txt"), 'r', encoding="utf-8") as ff:
            relations_dom_ran_lines = ff.readlines()
            for line_relation_dom_ran in tqdm(relations_dom_ran_lines):
                temp_relation_dom_ran = line_relation_dom_ran.strip().split('\t')
                relations_dom_ran.append(Relation_Dom_Ran(relation=temp_relation_dom_ran[0], dom=temp_relation_dom_ran[1], rang=temp_relation_dom_ran[2]))               
        relations_dom_ran=CalcularBernoulliDistribution(relations_dom_ran, lines_str_set)
        
        examples = []
        for (i, line) in enumerate(lines):
         
            print("******i", i)
            head_ent_text = ent2text[line[0]]
            tail_ent_text = ent2text[line[2]]
            relation_text = rel2text[line[1]]
            guidP = "%s-%s" % (set_type, i)

            #Corrupting head or tail according to GoodNegativeSampler with Bernoulli
            headOrTail=["head","tail"] 
            headProbability, tailProbability=0,0
                        
            prs=[relation_dom_ran_aux.pr for relation_dom_ran_aux in relations_dom_ran if relation_dom_ran_aux.relation==line[1]]
            if len(prs)>0:
                headProbability=prs[0]
                tailProbability=1-headProbability
            rnd=random.choices(headOrTail, weights=(headProbability, tailProbability), k=1)
                        
            guidN = "%s-%s" % (set_type + "_corrupt", i) 
            good_triple_flag=False

            if rnd[0]=="head":
                tmp_head = ''
                cant=0
                while True:
                    cant=cant+1
                    tmp_ent_list = set(entities)
                    tmp_ent_list.remove(line[0])
                    tmp_ent_list = list(tmp_ent_list)
                    tmp_head = random.choice(tmp_ent_list)
                    tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                    if tmp_triple_str not in lines_str_set:
                        doms=[relation_dom_ran_aux.dom for relation_dom_ran_aux in relations_dom_ran if relation_dom_ran_aux.relation==line[1] ]
                        if len(doms)>0:
                            good_triple_flag=validarTripleta(sujeto=tmp_head, dom=doms[0], obj=None, ran=None, tripletas=lines_str_set, relations=relations_dom_ran)
                        if good_triple_flag:
                            break
                        if cant==800:
                            break
                if good_triple_flag:
                    examples.append(InputExample(guid=guidP, text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text, label="1"))
                    train_with_corrupts.append(Triple(guid=guidP,subject= line[0], predicate=line[1], obj=line[2], label="1"))

                    tmp_head_text = ent2text[tmp_head]       
                    examples.append(InputExample(guid=guidN, text_a=tmp_head_text, text_b=relation_text, text_c = relation_text, label="0")) 
                    train_with_corrupts.append(Triple(guid=guidN,subject=tmp_head, predicate=line[1], obj=line[2], label="0"))
            else:
                tmp_tail = ''
                cant=0
                while True:
                    cant=cant+1
                    tmp_ent_list = set(entities)
                    tmp_ent_list.remove(line[2])
                    tmp_ent_list = list(tmp_ent_list)
                    tmp_tail = random.choice(tmp_ent_list)
                    tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                    if tmp_triple_str not in lines_str_set:
                        rangs=[relation_dom_ran_aux.rang for relation_dom_ran_aux in relations_dom_ran if relation_dom_ran_aux.relation==line[1] ]
                        if len(rangs)>0:
                            good_triple_flag=validarTripleta(sujeto=None, dom=None, obj=tmp_tail, ran=rangs[0], tripletas=lines_str_set, relations=relations_dom_ran)
                        if good_triple_flag:
                            break
                        if cant==800:
                            break
                if good_triple_flag:
                    examples.append(InputExample(guid=guidP, text_a=head_ent_text, text_b=relation_text, text_c = tail_ent_text, label="1"))
                    train_with_corrupts.append(Triple(guid=guidP,subject= line[0], predicate=line[1], obj=line[2], label="1"))

                    tmp_tail_text = ent2text[tmp_tail]
                    examples.append(InputExample(guid=guidN, text_a=head_ent_text, text_b=relation_text, text_c = tmp_tail_text, label="0"))  
                    train_with_corrupts.append(Triple(guid=guidN,subject= line[0] , predicate=line[1], obj=tmp_tail, label="0"))
        
        with open(examples_file, "w", encoding="utf-8") as writer:
                for sample in examples:
                    writer.write("%s\t%s\t%s\t%s\t%s\n" % (sample.guid, sample.text_a, sample.text_b, sample.text_c, sample.label))
        
        #Generando los ejemplos a utilizar en entrenamiento
        with open(train_with_corrupts_file, "w", encoding="utf-8") as writer:
                for triple in train_with_corrupts:
                   writer.write("%s\t%s\t%s\t%s\n" % (triple.subject, triple.predicate, triple.obj, triple.label))
        return examples 

In [ ]:
def main():
        
    arg_dict ={
        "task_name": "kg",
        "data_dir": data_dir,      
        }
    processors = {
        "kg": KGProcessor,
        }  
    task_name = arg_dict["task_name"].lower()
    processor = processors[task_name]()
 
    train_examples = processor.get_train_examples(arg_dict["data_dir"])   
    print("len(train_examples)",len(train_examples))  

In [ ]:
main()  